In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print(X_train.shape)

In [ ]:
PATH_FOR_CSV:str = "/kaggle/input/chinese-mnist/chinese_mnist.csv"

c_mnist = pd.read_csv(PATH_FOR_CSV)
c_mnist = c_mnist.drop(columns=['character'], axis = 1)
c_mnist.head()

In [ ]:
TEST_INDEX:int = 0
MAX:int = 10
RANDOM_STATE:int = 40
MNIST_X_NUM_ROWS:int = mnist_X.shape[0]
TEST_SIZE:float = 0.1
TRAIN_EPOCHS:int = 10
TRANSFER_TRAIN_EPOCHS:int = 5
VERBOSE:int = 1
X_TRAIN_NUM_ROWS:int = X_train.shape[0]
X_TEST_NUM_ROWS:int = X_test.shape[0]
HOME_DIR:str = '/kaggle/input'

In [ ]:
# Simple feature adjustments

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = X_train.reshape(X_TRAIN_NUM_ROWS, 28, 28, 1)
X_test = X_test.reshape(X_TEST_NUM_ROWS, 28, 28, 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization

model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape = X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(28, kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=X_train,y=y_train, epochs=TRAIN_EPOCHS, verbose = VERBOSE)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
c_mnist['path'] = c_mnist['sample_id']
for index, row in c_mnist.iterrows():
    c_mnist['path'][index] = "/kaggle/input/chinese-mnist/data/data/input_" + str(c_mnist['suite_id'][index]) + "_" + str(c_mnist['sample_id'][index]) + "_" + str(c_mnist['code'][index]) + ".jpg"
c_mnist['path']

In [ ]:
print(c_mnist['path'][TEST_INDEX])

In [ ]:
mnist_X = []
mnist_y = []

for index, row in c_mnist.iterrows():
    if(c_mnist['value'][index] % MAX != 0):
        y = np.asarray(Image.open(c_mnist['path'][index]).resize((28, 28)))
        mnist_X.append(y)
        mnist_y.append(c_mnist['value'][index])
        
mnist_X = np.asarray(mnist_X)
mnist_y = np.asarray(mnist_y)

print(mnist_X.shape)
print(mnist_y.shape)
print(X_train.shape)
print(y_train.shape)

In [ ]:
print(type(X_train))
print(mnist_X[TEST_INDEX].shape)

In [ ]:
print(type(y_train[TEST_INDEX]))
print(type(c_mnist['value'][TEST_INDEX]))

In [ ]:
for layer in model.layers[:2]:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
mnist_X = mnist_X.reshape(MNIST_X_NUM_ROWS, 28, 28, 1)
mnist_X_train, mnist_X_test, mnist_y_train, mnist_y_test = train_test_split(mnist_X, mnist_y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
model.fit(x=mnist_X_train,y=mnist_y_train, epochs=TRANSFER_TRAIN_EPOCHS, verbose = VERBOSE)

In [ ]:
model.evaluate(x=mnist_X_test, y=mnist_y_test)